In [1]:
import pickle
import os
from datetime import datetime 
import pandas as pd
import numpy as np

In [2]:
# load the model

In [3]:
model_path = r"C:\Users\Digvijay Mohite\PYTHON & DATA SCIENCE\Present\Seoul_Bike_Sharing_Demand_Prediction\Models\xgboost_regressor_r2_0_936_v1.pkl"
model = pickle.load(open(model_path, "rb"))

In [4]:
data = [
    [-0.65432992,  1.24504573,  0.23697162, -0.5001855 ,  0.56154562,
       -0.28664608, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
        0.94701966,  0.14260346,  0.30459857, -0.57427456,  1.71961809,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
        2.49494334, -0.41367015],  
    
    [0.50060987,  0.98718632, -0.69180254,  2.40011233,  0.92262892,
        1.56419996, -0.12916836, -0.17535463, -0.22754885,  0.18254726,
       -0.98911087,  0.72160984,  0.30459857, -0.57427456, -0.58152447,
       -0.57866842, -0.40630934, -0.40916536, -0.4075004 , -0.40511677,
       -0.40081071, -0.41367015]
]

In [5]:
prediction = model.predict(data)
list(prediction)

[1230.4951, 1579.3689]

In [6]:
# 1213,  1391

### Load standard scalar

In [7]:
sc_dump_path = r"C:\Users\Digvijay Mohite\PYTHON & DATA SCIENCE\Present\Seoul_Bike_Sharing_Demand_Prediction\Models\sc.pkl"

sc = pickle.load(open(sc_dump_path, "rb"))

In [8]:
sc

StandardScaler()

### User Input

In [9]:
Date = "10/06/2023"
Hour = 21
Temperature = 25
Humidity = 55
Wind_speed = 67
Visibility = 25
Solar_Radiation = 0.0
Rainfall = 0.7
Snowfall = 0.2
Seasons = "Winter"
Holiday = "No Holiday"
Functioning_Day = "Yes"

#### coverting user data into model consumable format

In [10]:
holiday_dic = {"No Holiday":0,  "Holiday":1}
functioning_day = {"No":0, "Yes":1}

def get_string_to_datetime(date):
    dt = datetime.strptime(Date, "%d/%m/%Y")
    return {"day":dt.day, "month":dt.month, "year":dt.year, "week_day":dt.strftime("%A")}

str_to_date = get_string_to_datetime(Date)
str_to_date

{'day': 10, 'month': 6, 'year': 2023, 'week_day': 'Saturday'}

In [11]:
user_input_list = [Hour, Temperature , Humidity , Wind_speed ,Visibility,
                   Solar_Radiation , Rainfall , Snowfall , holiday_dic[Holiday] ,functioning_day[Functioning_Day],
                   str_to_date['day'], str_to_date['month'], str_to_date['year']]

features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)','Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)',
       'Holiday', 'Functioning Day', 'Day', 'Month', 'Year']

df_user_input = pd.DataFrame([user_input_list], columns=features_name)
df_user_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,21,25,55,67,25,0.0,0.7,0.2,0,1,10,6,2023


In [12]:
user_input_list

[21, 25, 55, 67, 25, 0.0, 0.7, 0.2, 0, 1, 10, 6, 2023]

In [13]:
def seasons_to_df(Seasons):
    seasons_cols = ['Spring', 'Summer','Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype=int)

    df_seasons = pd.DataFrame(seasons_data, columns= seasons_cols)
    if Seasons in seasons_cols:
        df_seasons[Seasons] = 1
    return df_seasons
    
df_season = seasons_to_df(Seasons)
df_season

,Spring,Summer,Winter
0,0,0,1


In [14]:
def days_df(week_day):
    days_names = ['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_name_data = np.zeros((1, len(days_names)),dtype = 'int')
    df_days = pd.DataFrame(days_name_data, columns=days_names)

    if week_day in days_names:
        df_days[week_day]=1
    return df_days

df_days = days_df(str_to_date['week_day'])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [15]:
str_to_date['week_day']

'Saturday'

In [16]:
days_df('Friday')

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,0,0,0


In [17]:
df_for_pred = pd.concat([df_user_input, df_season, df_days], axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,21,25,55,67,25,0.0,0.7,0.2,0,1,...,2023,0,0,1,0,1,0,0,0,0


In [19]:
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 1.36681471,  1.01214045, -0.15409119, 63.11301355, -2.33371591,
        -0.65681529,  0.46358031,  0.29083855, -0.22754885,  0.18254726,
        -0.64744078, -0.14689974, 18.24263884, -0.57427456, -0.58152447,
         1.72810536, -0.40630934,  2.44399966, -0.4075004 , -0.40511677,
        -0.40081071, -0.41367015]])

### Prediction In Deployment

In [23]:
model.predict(sc_data_for_pred)

array([696.53235], dtype=float32)

In [26]:
print(f"Rented Bike Demand on date :{Date} and Time:{Hour} is :{round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented Bike Demand on date :10/06/2023 and Time:21 is :697
